In [1]:
from finance_complaint.entity.spark_manager import spark_session
from finance_complaint.ml.feature import FrequencyImputer
from finance_complaint.entity.complaint_column import ComplaintColumn
file_path="finance_artifact/data_ingestion/feature_store/finance_complaint"
df = spark_session.read.parquet(file_path)


22/09/09 12:53:42 WARN Utils: Your hostname, avnish-yadav resolves to a loopback address: 127.0.1.1; using 192.168.3.35 instead (on interface wlo1)
22/09/09 12:53:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/09 12:53:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import col,when,regexp_replace

In [9]:
from finance_complaint.ml.feature import FrequencyEncoder

In [10]:
df.columns

['company',
 'company_public_response',
 'company_response',
 'complaint_id',
 'complaint_what_happened',
 'consumer_consent_provided',
 'consumer_disputed',
 'date_received',
 'date_sent_to_company',
 'issue',
 'product',
 'state',
 'sub_issue',
 'sub_product',
 'submitted_via',
 'tags',
 'timely',
 'zip_code']

In [11]:
in_col = ['timely']
out_col=['o_company']

In [12]:
fe=FrequencyEncoder(inputCols=in_col,outputCols=out_col)
model=fe.fit(df)

In [13]:
df = model.transform(df)

In [14]:
df[out_col].show()

+---------+
|o_company|
+---------+
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
|  2738212|
+---------+
only showing top 20 rows



In [11]:
data={"Yes":str(0),"No":str(1)}

In [ ]:
df.withColumn('r_timely',cast(when(col('timely').endswith('s'),regexp_replace('timely',"Yes",str(1))))).

In [12]:
df.withColumn('r_timely',cast(when(col('timely').endswith('s'),regexp_replace('timely',"Yes",str(1))))).

+------+--------+
|timely|r_timely|
+------+--------+
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
|   Yes|       1|
+------+--------+
only showing top 20 rows



In [18]:
df.join(df).count()

ERROR:root:KeyboardInterrupt while sending command.               (0 + 8) / 100]
Traceback (most recent call last):
  File "/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/env/venv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/env/venv/lib/python3.8/site-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/avnish/iNeuron_Private_Intelligence_Limited/industry_ready_project/env/venv/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [45]:
df.groupby("timely").count().collect()[0]['timely']

'No'

In [3]:
cols = ComplaintColumn()
df.filter(f'{cols.col_sub_product} is null').count()
df.filter(f'{cols.col_state} is null').count()

40075

In [4]:
in_col = [cols.col_state,cols.col_sub_product]
out_col = [f"o_{in_c}" for in_c in in_col ]
out_col

['o_state', 'o_sub_product']

In [5]:
fi = FrequencyImputer(inputCols=in_col,outputCols=out_col)
model = fi.fit(dataset=df)

{'inputCols': ['state', 'sub_product'], 'outputCols': ['o_state', 'o_sub_product']}


In [6]:
df = model.transform(df)

['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state']
o_state state
['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_sub_product sub_product


In [21]:
df.filter(f'o_{cols.col_sub_product} is null').count()
df.filter(f'o_{cols.col_state} is null').count()

0

In [13]:
df.filter(f"u_state is null").count()

0

In [10]:
from pyspark.ml.pipeline import Pipeline

In [11]:
pipeline = Pipeline(stages=[fi])

In [12]:
model = pipeline.fit(df)

['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_state state
['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_sub_product sub_product


In [13]:
df=model.transform(df)

['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_state state
['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_sub_product sub_product


In [14]:
df.printSchema()

root
 |-- company: string (nullable = true)
 |-- company_public_response: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- complaint_id: string (nullable = true)
 |-- complaint_what_happened: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- sub_issue: string (nullable = true)
 |-- sub_product: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- o_state: string (nullable = false)
 |-- o_sub_product: string (nullable = false)



In [82]:
model.transform(df)

['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state']
o_state state
['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_sub_product sub_product


DataFrame[company: string, company_public_response: string, company_response: string, complaint_id: string, complaint_what_happened: string, consumer_consent_provided: string, consumer_disputed: string, date_received: string, date_sent_to_company: string, issue: string, product: string, state: string, sub_issue: string, sub_product: string, submitted_via: string, tags: string, timely: string, zip_code: string, o_state: string, o_sub_product: string, oconsumer_consent_provided: string]

In [16]:
model.save('transformation_pipeline')

In [18]:
from pyspark.ml.pipeline import PipelineModel

In [19]:
model = PipelineModel.load("transformation_pipeline")

{}


In [20]:
model.transform(df)

['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_state state
['company', 'company_public_response', 'company_response', 'complaint_id', 'complaint_what_happened', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'sub_issue', 'sub_product', 'submitted_via', 'tags', 'timely', 'zip_code', 'o_state', 'o_sub_product']
o_sub_product sub_product


DataFrame[company: string, company_public_response: string, company_response: string, complaint_id: string, complaint_what_happened: string, consumer_consent_provided: string, consumer_disputed: string, date_received: string, date_sent_to_company: string, issue: string, product: string, state: string, sub_issue: string, sub_product: string, submitted_via: string, tags: string, timely: string, zip_code: string, o_state: string, o_sub_product: string]

In [31]:
for data in {"a":"as"}:
    print(data)

a
